In [ ]:
!nvidia-smi

In [9]:
%matplotlib inline


# uso y pruebas

In [10]:
!python classifier.py --help
# !python classifier_bert.py --help
# !python classifier.py --help

Traceback (most recent call last):
  File "classifier.py", line 20, in <module>
    from embedding import load_embedding_label
  File "/home/baruj/Documentos/proyectos/LIMU-BERT-Public/embedding.py", line 17, in <module>
    from plot import plot_reconstruct_sensor, plot_embedding
  File "/home/baruj/Documentos/proyectos/LIMU-BERT-Public/plot.py", line 11, in <module>
    import seaborn as sn
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/__init__.py", line 6, in <module>
    from .rcmod import *
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/rcmod.py", line 7, in <module>
    from . import palettes, _orig_rc_params
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/palettes.py", line 9, in <module>
    from .utils import desaturate, set_hls_values, get_color_cycle
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/utils.py", line 10, in <module>
    impor

In [11]:
!python embedding.py v1 uci 20_120 -f uci

Traceback (most recent call last):
  File "embedding.py", line 17, in <module>
    from plot import plot_reconstruct_sensor, plot_embedding
  File "/home/baruj/Documentos/proyectos/LIMU-BERT-Public/plot.py", line 11, in <module>
    import seaborn as sn
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/__init__.py", line 6, in <module>
    from .rcmod import *
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/rcmod.py", line 7, in <module>
    from . import palettes, _orig_rc_params
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/palettes.py", line 9, in <module>
    from .utils import desaturate, set_hls_values, get_color_cycle
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/seaborn/utils.py", line 10, in <module>
    import matplotlib.pyplot as plt
  File "/home/baruj/miniconda3/envs/limu-py38/lib/python3.8/site-packages/matplotlib/pyplot.py", line 2282, in <mod

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install tensorflow
!pip install tensorboard

In [ ]:
!python pretrain.py v1 uci 20_120 -s limu_v1 

In [ ]:
!python embedding.py v1 uci 20_120 -f limu_v1


In [ ]:
!python embedding.py "BERT" "hhar" "20_120" -g 0

# utils.py linea 411 msj de erro

# carga desede codigo gpt

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import os
import json
import plot 
import utils 
import torch
import torch.nn as nn

In [ ]:
class LIMUBERTModel(nn.Module):
    def __init__(self):
        super(LIMUBERTModel, self).__init__()
        self.embed = nn.Linear(6, 72)  # Adjusted to match the saved model
        self.transformer = nn.Transformer(d_model=72, nhead=8, num_encoder_layers=6)  # Adjusted to match the saved model
        self.fc = nn.Linear(72, 72)  # Adjusted to match the saved model

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

    def get_embeddings(self, x):
        x = self.embed(x)
        return x

# Instantiate the model
model = LIMUBERTModel()

# Path to your model file
model_path = 'saved/pretrain_base_hhar_20_120/hhar.pt'

# Load the state dictionary
state_dict = torch.load(model_path)

# Load the state dict into the model
model.load_state_dict(state_dict, strict=False)

# Set model to evaluation mode
model.eval()


In [ ]:
# Load dataset
data_path = "dataset/hhar/data_20_120.npy"
labels_path = 'dataset/hhar/label_20_120.npy'

data = np.load(data_path)
labels = np.load(labels_path)

# Flatten the data
data_flat = data.reshape(data.shape[0], -1)



# Extract embeddings from the flattened data
embeddings = get_embeddings(model, data_flat)

# Ensure labels are correctly shaped
if len(labels.shape) > 1 and labels.shape[1] > 1:
    labels = labels[:, 0]

# Function to visualize embeddings
def visualize_embeddings(embeddings, labels):
    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 10))
    scatter = plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=labels, cmap='viridis', alpha=0.5)
    plt.colorbar(scatter)
    plt.title("t-SNE visualization of embeddings")
    plt.xlabel("t-SNE component 1")
    plt.ylabel("t-SNE component 2")
    plt.show()

# Inline plotting for Jupyter Notebook
%matplotlib inline

# Visualize the embeddings
visualize_embeddings(state_dict, labels)
